In [3]:
import os
os.environ['PL_API_KEY'] = str('your-key')

In [4]:
#specify the filter

# the geo json geometry object we got from geojson.io
geo_json_geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [
              -121.72562956809998,
              46.77024032708875
            ],
            [
              -121.72287225723267,
              46.77024032708875
            ],
            [
              -121.72287225723267,
              46.77120296608276
            ],
            [
              -121.72562956809998,
              46.77120296608276
            ],
            [
              -121.72562956809998,
              46.77024032708875
            ]
    ]
  ]
}

# filter for items the overlap with our chosen geometry
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geo_json_geometry
}

# filter images acquired in a certain date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2018-07-01T00:00:00.000Z",
    "lte": "2018-08-01T00:00:00.000Z"
  }
}

# filter any images which are more than 50% clouds
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# create a filter that combines our geo and date filters
# could also use an "OrFilter"
plot_1 = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [5]:
#gets the stats - i.e. avaliable images in the timeframe
import os
import requests
from requests.auth import HTTPBasicAuth



# Stats API request object
stats_endpoint_request = {
  "interval": "day",
  "item_types": ["REOrthoTile"],
  "filter": plot_1
}

# fire off the POST request
result = \
  requests.post(
    'https://api.planet.com/data/v1/stats',
    auth=HTTPBasicAuth(os.environ['PL_API_KEY'], ''),
    json=stats_endpoint_request)

print (result.text)

{"utc_offset": "+0h", "interval": "day", "buckets": [{"count": 1, "start_time": "2018-07-09T00:00:00.000000Z"}, {"count": 1, "start_time": "2018-07-14T00:00:00.000000Z"}, {"count": 1, "start_time": "2018-07-27T00:00:00.000000Z"}]}


In [6]:
#now lets get the metadata
# fire off the POST request
result_metadata = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(os.environ['PL_API_KEY'], ''),
    json=stats_endpoint_request)

In [7]:
print(result_metadata.text)

{"_links": {"_first": "https://api.planet.com/data/v1/searches/8db4414188e7486da1fd2d38af60dd35/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D", "_next": "https://api.planet.com/data/v1/searches/8db4414188e7486da1fd2d38af60dd35/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6ICIyMDE4LTA3LTEwVDA5OjI4OjQ4LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMTgwNzA5XzE5MTI0OV8xMDYwNjE5X1JhcGlkRXllLTIiLCAic29ydF9kZXNjIjogdHJ1ZX0%3D", "_self": "https://api.planet.com/data/v1/searches/8db4414188e7486da1fd2d38af60dd35/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D"}, "features": [{"_links": {"_s

In [8]:
import json
jstl= json.loads(result_metadata.text)

In [9]:
for feature in jstl['features']:
    print(feature['id'])

20180727_190839_1060619_RapidEye-1
20180714_191742_1060619_RapidEye-2
20180709_191249_1060619_RapidEye-2


In [10]:
#Depending on the amount of Assets you want to activate, 
#the activation step can take time to complete. A best practice is to activate your desired items and then
# periodically check the status until it becomes "active".

In [11]:
import os
import requests

item_id = "20180727_190839_1060619_RapidEye-1"
item_type = "REOrthoTile"
asset_type = "visual"

# setup auth
session = requests.Session()
session.auth = (os.environ['PL_API_KEY'], '')

# request an item
item = \
  session.get(
    ("https://api.planet.com/data/v1/item-types/" +
    "{}/items/{}/assets/").format(item_type, item_id))

# extract the activation url from the item for the desired asset
item_activation_url = item.json()[asset_type]["_links"]["activate"]

# request activation
response = session.post(item_activation_url)

print(response.status_code)

204


In [12]:
#status of asset
print(item.json())


{'analytic': {'_links': {'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzI3XzE5MDgzOV8xMDYwNjE5X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljIiwgImN0IjogIml0ZW0tdHlwZSJ9', 'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzI3XzE5MDgzOV8xMDYwNjE5X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljIiwgImN0IjogIml0ZW0tdHlwZSJ9/activate', 'type': 'https://api.planet.com/data/v1/asset-types/analytic'}, '_permissions': ['download'], 'md5_digest': None, 'status': 'inactive', 'type': 'analytic'}, 'analytic_xml': {'_links': {'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzI3XzE5MDgzOV8xMDYwNjE5X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljX3htbCIsICJjdCI6ICJpdGVtLXR5cGUifQ', 'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTgwNzI3XzE5MDgzOV8xMDYwNjE5X1JhcGlkRXllLTEiLCAiYyI6ICJSRU9ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljX3htbCIsICJjdCI6ICJpdGVtLXR5cGUifQ/activate', 'type': 'https://api.p

In [13]:
item.json()[asset_type]["status"]

'active'

In [14]:
# request status of an item
item = \
  session.get(
    ("https://api.planet.com/data/v1/item-types/" +
    "{}/items/{}/assets/").format(item_type, item_id))

In [15]:
item_id

'20180727_190839_1060619_RapidEye-1'

In [16]:
item.json()[asset_type]["status"]

'active'

In [17]:
#status is active, so lets download
CLIP_API_URL = "https://api.planet.com/compute/ops/clips/v1/"
PL_API_KEY = os.environ["PL_API_KEY"]
from retrying import retry

In [18]:
@retry(wait_fixed=5000)
def check_clip_op(id):
    r = requests.get("{_base}/{id}".format(_base = CLIP_API_URL, id=id), auth=(PL_API_KEY, ""))
    if r.json()['state'] != "succeeded":
        raise Exception("Not Yet")
        print("...waiting")
    else:
        print("response found.")
        return(r.json())

In [19]:
payload = {
    "aoi" : geo_json_geometry,
    "targets" : [{
        "item_id" : item_id, 
        "item_type" : "REOrthoTile", 
        "asset_type" : 'visual'
    }]
}

r = requests.post(CLIP_API_URL, auth=(PL_API_KEY, ""), json=payload)

check_clip_op(r.json()['id'])

response found.


{'_links': {'_self': 'https://api.planet.com/compute/ops/clips/v1/0571c25f-d715-4856-80fa-90c73a834601',
  'results': ['https://api.planet.com/data/v1/download?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1Mzg5OTk0OTIsInN1YiI6Ind1VVh6S1VMdUhNRmlETkFFdXg2bWNEcTdKc0dPR1ZMMmJ6a3ZEZHJHcm5USXBUSitldlZoUEZNZ21MMjlQL0NxaVpnT01NREdPVm0xSllFOGNOZVVnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiJ7XCJ0eXBlXCI6IFwiUG9seWdvblwiLCBcImNvb3JkaW5hdGVzXCI6IFtbWy0xMjEuNzI1NjI5NTY4MDk5OTgsIDQ2Ljc3MDI0MDMyNzA4ODc1XSwgWy0xMjEuNzIyODcyMjU3MjMyNjcsIDQ2Ljc3MDI0MDMyNzA4ODc1XSwgWy0xMjEuNzIyODcyMjU3MjMyNjcsIDQ2Ljc3MTIwMjk2NjA4Mjc2XSwgWy0xMjEuNzI1NjI5NTY4MDk5OTgsIDQ2Ljc3MTIwMjk2NjA4Mjc2XSwgWy0xMjEuNzI1NjI5NTY4MDk5OTgsIDQ2Ljc3MDI0MDMyNzA4ODc1XV1dfSIsImFzc2V0cyI6W3siaXRlbV90eXBlIjoiUkVPcnRob1RpbGUiLCJhc3NldF90eXBlIjoidmlzdWFsIiwiaXRlbV9pZCI6IjIwMTgwNzI3XzE5MDgzOV8xMDYwNjE5X1JhcGlkRXllLTEifV0sInVybCI6Imh0dHBzOi8vcGxhbmV0LXBsYXRmb3JtLWNvbXB1dGVkLWFzc2V0cy5zMy11cy13ZXN0LTIuYW1hem9uYXdzLmNvbS8wN

In [20]:
plot_1_image = check_clip_op(r.json()['id'])

response found.


In [21]:
plot_1_image

{'_links': {'_self': 'https://api.planet.com/compute/ops/clips/v1/0571c25f-d715-4856-80fa-90c73a834601',
  'results': ['https://api.planet.com/data/v1/download?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1Mzg5OTk0OTMsInN1YiI6IldXVi8rRk9LUy9YQ0RjQWJWT0JpeWkrbmhmc1ArZHVzOXo0aUxRZ0Y3MjQ1bUg4VkRDdDVSYzROeWRjWEZ4TlNKLzZJeEhkQlpzNUd1OUYwOGJzMEF3PT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiJ7XCJ0eXBlXCI6IFwiUG9seWdvblwiLCBcImNvb3JkaW5hdGVzXCI6IFtbWy0xMjEuNzI1NjI5NTY4MDk5OTgsIDQ2Ljc3MDI0MDMyNzA4ODc1XSwgWy0xMjEuNzIyODcyMjU3MjMyNjcsIDQ2Ljc3MDI0MDMyNzA4ODc1XSwgWy0xMjEuNzIyODcyMjU3MjMyNjcsIDQ2Ljc3MTIwMjk2NjA4Mjc2XSwgWy0xMjEuNzI1NjI5NTY4MDk5OTgsIDQ2Ljc3MTIwMjk2NjA4Mjc2XSwgWy0xMjEuNzI1NjI5NTY4MDk5OTgsIDQ2Ljc3MDI0MDMyNzA4ODc1XV1dfSIsImFzc2V0cyI6W3siaXRlbV90eXBlIjoiUkVPcnRob1RpbGUiLCJhc3NldF90eXBlIjoidmlzdWFsIiwiaXRlbV9pZCI6IjIwMTgwNzI3XzE5MDgzOV8xMDYwNjE5X1JhcGlkRXllLTEifV0sInVybCI6Imh0dHBzOi8vcGxhbmV0LXBsYXRmb3JtLWNvbXB1dGVkLWFzc2V0cy5zMy11cy13ZXN0LTIuYW1hem9uYXdzLmNvbS8wN

In [22]:
download_url = plot_1_image['_links']['results'][0]

In [23]:
download_url

'https://api.planet.com/data/v1/download?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1Mzg5OTk0OTMsInN1YiI6IldXVi8rRk9LUy9YQ0RjQWJWT0JpeWkrbmhmc1ArZHVzOXo0aUxRZ0Y3MjQ1bUg4VkRDdDVSYzROeWRjWEZ4TlNKLzZJeEhkQlpzNUd1OUYwOGJzMEF3PT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiJ7XCJ0eXBlXCI6IFwiUG9seWdvblwiLCBcImNvb3JkaW5hdGVzXCI6IFtbWy0xMjEuNzI1NjI5NTY4MDk5OTgsIDQ2Ljc3MDI0MDMyNzA4ODc1XSwgWy0xMjEuNzIyODcyMjU3MjMyNjcsIDQ2Ljc3MDI0MDMyNzA4ODc1XSwgWy0xMjEuNzIyODcyMjU3MjMyNjcsIDQ2Ljc3MTIwMjk2NjA4Mjc2XSwgWy0xMjEuNzI1NjI5NTY4MDk5OTgsIDQ2Ljc3MTIwMjk2NjA4Mjc2XSwgWy0xMjEuNzI1NjI5NTY4MDk5OTgsIDQ2Ljc3MDI0MDMyNzA4ODc1XV1dfSIsImFzc2V0cyI6W3siaXRlbV90eXBlIjoiUkVPcnRob1RpbGUiLCJhc3NldF90eXBlIjoidmlzdWFsIiwiaXRlbV9pZCI6IjIwMTgwNzI3XzE5MDgzOV8xMDYwNjE5X1JhcGlkRXllLTEifV0sInVybCI6Imh0dHBzOi8vcGxhbmV0LXBsYXRmb3JtLWNvbXB1dGVkLWFzc2V0cy5zMy11cy13ZXN0LTIuYW1hem9uYXdzLmNvbS8wNTcxYzI1Zi1kNzE1LTQ4NTYtODBmYS05MGM3M2E4MzQ2MDEtY2xpcHMuemlwP1gtQW16LUFsZ29yaXRobT1BV1M0LUhNQUMtU0hBMjU2XHUwMDI2WC1BbXot

In [24]:
url_template = "{id}_download.zip"
local_url = url_template.format(id=r.json()['id'])

In [25]:
local_url

'0571c25f-d715-4856-80fa-90c73a834601_download.zip'

In [26]:
def download_file(url, dest):
    local_filename = dest
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True, auth=(PL_API_KEY, ""))
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename

In [27]:
download_file(download_url, local_url)

'0571c25f-d715-4856-80fa-90c73a834601_download.zip'